<a href="https://colab.research.google.com/github/rishike/langchain/blob/master/tool_calling_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from getpass import getpass
os.environ["GOOGLE_API_KEY"] =  getpass("Provide your Google API key here")

Provide your Google API key here··········


In [2]:
!pip install -q google-ai-generativelanguage langchain-google-genai langchain-core requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 31.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [4]:
@tool
def multiply(a: int, b: int) -> int:
  """ Given 2 number a and b return thier product"""
  return a * b

In [5]:
print(multiply.invoke({'a': 3, 'b': 4}))

12


In [6]:
multiply.name


'multiply'

In [7]:
# tool binding

In [16]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

In [17]:
llm_with_tools = llm.bind_tools([multiply])

In [18]:
query = HumanMessage('can you multiply 3 with 10')
messages = [query]

In [19]:
messages

[HumanMessage(content='can you multiply 3 with 10', additional_kwargs={}, response_metadata={})]

In [20]:
llm_with_tools

RunnableBinding(bound=ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), temperature=0.0, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x781f21f57390>, default_metadata=(), model_kwargs={}), kwargs={'tools': [{'type': 'function', 'function': {'name': 'multiply', 'description': 'Given 2 number a and b return thier product', 'parameters': {'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object'}}}]}, config={}, config_factories=[])

In [21]:
llm_with_tools.invoke('Hi how are you')

AIMessage(content="I'm doing well, thank you for asking! How are you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run--4a5b4f29-9442-4e0b-a58b-45aae37a0ec0-0', usage_metadata={'input_tokens': 22, 'output_tokens': 17, 'total_tokens': 39, 'input_token_details': {'cache_read': 0}})

In [22]:
result = llm_with_tools.invoke('can you multiply 3 with 10')

In [23]:
llm_with_tools.invoke('can you multiply 3 with 10').tool_calls[0]

{'name': 'multiply',
 'args': {'a': 3.0, 'b': 10.0},
 'id': '14a15917-3095-4579-84cc-7d41c18e70f1',
 'type': 'tool_call'}

Tool execution

In [24]:
result.tool_calls[0]['args']

{'a': 3.0, 'b': 10.0}

In [25]:
multiply.invoke(result.tool_calls[0]['args'])

30

In [26]:
multiply.invoke(result.tool_calls[0]['args'])

30

In [27]:
multiply.invoke(result.tool_calls[0])

ToolMessage(content='30', name='multiply', tool_call_id='6c32acf7-0ddd-4e0d-b01c-0373012b2c07')

In [28]:
messages.append(result)

In [29]:
tool_result = multiply.invoke(result.tool_calls[0])

In [30]:
messages.append(tool_result)

In [31]:
messages

[HumanMessage(content='can you multiply 3 with 10', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 3.0, "b": 10.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run--80bce7fb-4186-4763-b1f8-934617e5dd34-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 10.0}, 'id': '6c32acf7-0ddd-4e0d-b01c-0373012b2c07', 'type': 'tool_call'}], usage_metadata={'input_tokens': 27, 'output_tokens': 5, 'total_tokens': 32, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='30', name='multiply', tool_call_id='6c32acf7-0ddd-4e0d-b01c-0373012b2c07')]

In [32]:
llm_with_tools.invoke(messages).content

'The product of 3 and 10 is 30.'

**currency conversion tool**

In [33]:
from langchain_core.tools import InjectedToolArg
from typing import Annotated

In [34]:
# tool create

@tool
def get_conversation_tool(base_currency: str, target_currency:str) -> float:
  """ This function fetches the currency conversion factor between a given base currency and a target currency"""
  url = f"https://v6.exchangerate-api.com/v6/9554b7e135b4da24b2305433/pair/{base_currency}/{target_currency}"
  response = requests.get(url)
  return response.json()


@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """ This function converts the base currency value to the target currency value using the conversion rate"""
  return base_currency_value * conversion_rate

In [35]:
get_conversation_tool.invoke({'base_currency': 'USD', 'target_currency': 'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1747612801,
 'time_last_update_utc': 'Mon, 19 May 2025 00:00:01 +0000',
 'time_next_update_unix': 1747699201,
 'time_next_update_utc': 'Tue, 20 May 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 85.638}

In [36]:
convert.invoke({'base_currency_value': 10, 'conversion_rate': 85.6035})

856.035

In [54]:
# tool binding
conversion_llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)

In [55]:
conversion_llm_with_tools = llm.bind_tools([get_conversation_tool, convert])

In [56]:
messages = [HumanMessage('What is the conversion factor between USD and INR, and based on that can you convert 10 usd to inr')]

In [57]:
messages

[HumanMessage(content='What is the conversion factor between USD and INR, and based on that can you convert 10 usd to inr', additional_kwargs={}, response_metadata={})]

In [58]:
ai_message = conversion_llm_with_tools.invoke(messages)

In [59]:
ai_message

AIMessage(content="I need to define the base and target currencies first.  I'll assume the `get_conversation_tool` function will provide the conversion factor.  However, I cannot execute the function without knowing its implementation details.  Please provide the implementation of `get_conversation_tool` and `convert` functions.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run--6e890bb8-ab67-4310-b98a-a09fa92e0b91-0', usage_metadata={'input_tokens': 90, 'output_tokens': 65, 'total_tokens': 155, 'input_token_details': {'cache_read': 0}})

In [60]:
messages.append(ai_message)

In [61]:
import json

In [62]:
for tool_call in ai_message.tool_calls:
  if tool_call['name'] == 'get_convestion_factor':
    tool_message1 = get_conversation_tool.invoke(tool_call)
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    messages.append(tool_message1)

  if tool_call['name'] == 'convert':
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    print(tool_message2)
    messages.append(tool_message2)


In [63]:
result = conversion_llm_with_tools.invoke(messages)

In [64]:
result

AIMessage(content='Once you provide the implementation, I can proceed with the conversion.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run--a4deeaab-bd88-40d1-b761-c0a516270e6f-0', usage_metadata={'input_tokens': 154, 'output_tokens': 14, 'total_tokens': 168, 'input_token_details': {'cache_read': 0}})